# Самообучение: раскраска изображений (image colorization)

Выполнил Оганов Александр Артурович

## Используемые статьи

Перечислим статьи и кратко опишем статьи, которые использовались в работе
1. [Colorful Image Colorization](https://arxiv.org/abs/1603.08511) - постановка задачи раскраски изображений и сравнение задачи классификации цвета с задачей регрессии (предсказание цвета). Автор расматривает задачу предсказания каналов a, b по каналу L [Lab color space](https://en.wikipedia.org/wiki/CIELAB_color_space), то есть по черно-белому изображению построить цветное;

2. [Split-Brain Autoencoders: Unsupervised Learning by Cross-Channel Prediction](https://arxiv.org/abs/1611.09842) - обобщение задачи раскраски изображений на произвольные каналы, в качестве основной модели был взят автокодировщик;

3. [Analysis of Different Losses for Deep Learning Image Colorization](https://arxiv.org/abs/2204.02980) - исследование влияния функции потерь для задачи раскраски изображений, перечеслены основные подходы к решению (классифкация, регрессия, приближение распределений, использование GAN) задачи;

4. [The Unreasonable Effectiveness of Deep Features as a Perceptual Metric](https://arxiv.org/abs/1801.03924) - построение метрики (на основе нейросети), которая отражает похожесть изображений для человека, полученную метрику автор статьи называет lpips.

# Подход к решению поставленной задачи

Решать задачу будем на датасете Food101, выбор обоснован красочностью датасета и его размерами.

В качестве основной модели, следуя статье  [Split-Brain Autoencoders: Unsupervised Learning by Cross-Channel Prediction](https://arxiv.org/abs/1611.09842), будем использовать автокодировщик, а в качестве эмбедингов - его латетное пространство. Код для обучения модели представлен в ноутбуке "my colorization.ipynb", для удобства были использованы блоки из библиотеки diffusers, а именно [Encoder и Decoder](https://github.com/huggingface/diffusers/blob/main/src/diffusers/models/vae.py). Архитектура модели описана в файле "Autoencoder.py".

В качестве функции потерь была взята комбинция двух функций. В след за статьей [Colorful Image Colorization](https://arxiv.org/abs/1603.08511) мы использовали [smooth L1 loss](https://pytorch.org/docs/stable/generated/torch.nn.SmoothL1Loss.html) для попиксельного сравнения и lpips для схожести изображений.

Для оценки полученных представлений проведем следующий эксперимент на датасете Food101. Мы разделим обучающую выборку в соотношние 9:1 для обучения автокодировщика и классификатора соотвественно. Также из обучения автокодировщика исключим 20 классов из 101. Тогда наш эксперемент будет выглядить следующим образом:

Обучение автокодировщика на 81 классе. Обучение классификатора на 1/10  обучащей выборки в латентном пространстве автокодировщика и его сравнение с классификатором, который обучался на 1/10 обучающей выборки


Тем самым мы получим ответы на следующие вопросы:

1. Помогают ли выученные представления в достижении лучшего результата классификации при наличии маленькой обучающей выборки (Эксперимент 1)?

2. Насколько хорошо обобщает данные автокодировщик и влияет ли на результаты классы, которые исполльзовались в его обучении?



Так как нашей целью не является получение высокого качества классификации, мы будем использовать простой сверточный классификатор ResNetFoodClassifierSmall:

```python

class BasicBlock(nn.Module):
    def __init__(self, in_channels, hidden, out_channels):
        super().__init__()
        self.basic_block = nn.Sequential(
            nn.Conv2d(in_channels, hidden, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(hidden, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
    
    def forward(self, x):
        return self.basic_block(x)

class ResNetFoodClassifierSmall(nn.Module):
    def __init__(self, prefix, in_channels, classes):
        super().__init__()
        
        self.back_bone_prefix = prefix
        
        for parametr in self.back_bone_prefix.parameters():
            parametr.requires_grad = False
            
        
        self.basic_block = BasicBlock(in_channels, 128, 256)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.out = nn.Linear(256, classes)
        
    def forward(self, x):
        x = self.back_bone_prefix(x)
        x = self.basic_block(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), x.size(1))
        x = self.out(x)
        
        return x
```

# Датасет

Датасет Foof101 содержит 101 класс разной еды, каждый класс содержит 750 экземпляров в обучающей выборке и 250 в валидационной. [Подробнее о датасете](https://huggingface.co/datasets/food101)

![alt Пример данных](img/example.png "Пример данных")

# Обучение автокодировщика

Параметры обучения:

```python
w1 = 1
w2 = 1

pixel_loss = nn.SmoothL1Loss()
perc_loss = lpip.LearnedPerceptualImagePatchSimilarity()

def loss_fn(x, y):
    l1 = w1 * pixel_loss(x, y)
    l2 = w2 * perc_loss(x, y)
    return l1 + l2, l1.item() / w1, l2.item() / w2

model = Autoencoder()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=2e-05)


```

Число параметров модели: 1287631, обучение проходило 16 эпох на 9/10 обучающей выборки за исключением 20 классов.

В качестве исключенных классов были выбраны: 'guacamole', 'spring_rolls', 'carrot_cake', 'paella', 'lobster_bisque', 'chicken_wings', 'ravioli', 'sashimi', 'peking_duck', 'peking_duck', 'scallops', 'tuna_tartare', 'churros', 'baklava', 'chocolate_cake', 'gyoza', 'baby_back_ribs', 'scallops', 'cup_cakes', 'filet_mignon'

Подробнее код продемонстриван в ноутбуке "my colorization.ipynb", история обучения и графики находятся по [ссылке](https://wandb.ai/3145tttt/colorization-ae/runs/q0xpt8s6/overview?workspace=user-3145tttt).

Ниже представлены примеры на обучающем датасете, на валидации и на 20 классах исключенных из обучения соответсвенно

<table align='left'>
    <tr>
    <td><img src="./img/train_vis.png" alt="Drawing" style="width: 300px;"/></td>
    <td><img src="./img/valid_vis.png" alt="Drawing" style="width: 300px;"/></td>
    <td><img src="./img/valid_out_vis.png" alt="Drawing" style="width: 300px;"/></td>
    </tr>
</table>

Общее качество раскрашенных изображений совпадает с исходной выборкой, но по цвету преобладают желтые тона. Так как нашей целью ялялось получение хорошо обученных представлений, мы не будем менять архитектуру и/или усложнять обучение.

# Обучение классификатаров

Будем обучать классификаторы отдельно на классах, которые использовались в обучение автокодировщика (на графиках in-domain), и на остальных (на графиках out-of-domain). В качестве классификатора будем использовать ResNetFoodClassifierSmall, для обучения на исходной выборке:
```python
model = ResNetFoodClassifierSmall(nn.Indentity(), 3, 101)

optimizer = optim.AdamW(model.parameters(), lr=0.0003, weight_decay=2e-05)
```

Для обучение на латентном пространстве (выученных представлениях):
```python
model = ResNetFoodClassifierSmall(autoencoder.encoder, 4, 101)

optimizer = optim.AdamW(model.parameters(), lr=0.0003, weight_decay=2e-05)
```

Число параметров модели: 325093, обучение проходило 10 эпох на 1/10 обучающей выборки. В качестве функции потерь будем использовать кросс-энтропию.

Код обучения находится в ноутбуке "classifier small.ipynb", история обучения [для in-domain](https://wandb.ai/3145tttt/colorization-classifier_small_in/overview?workspace=user-3145tttt) и [для out-of-domain](https://wandb.ai/3145tttt/colorization-classifier_small_out/overview?workspace=user-3145tttt)

<img src="./img/in_train_loss.png" alt="Drawing" style="width: 800px;"/>
<img src="./img/out_train_loss.png" alt="Drawing" style="width: 800px;"/>

На графиках видно, что классификатор обучаемый на латентном пространстве достигает меньших значений лосса в обоих случаях. На основании этого мы можем сделать вывод, что наш подход действительно повышает качество модели, то есть решает задачу самообучения, а именно автокодировщик выучивает нужные для классификации представления. 

# Выводы

На наш взгляд, основными выводами, которые можно извлечь из работы, ялвяются:

1. Использование lpips метрики и smooth L1 loss позволяет хорошо решать задачу раскраски изображений;
2. Описанный выше эксперимент продемонстрировал, что выученные представления повышают качество как на обучении так и на валидации;
3. Автокодировщик обладает хорошей обобщающей способностью и выученные представления повышают качество на классах, которые не использовались в обучении.

# Идеи

В рамках работы не удалось проверить некоторые идеи, которые не относятся к задаче самообучения на прямую, а именно:

1. Влияние функции потерь для классификации цвета из статьи [Colorful Image Colorization](https://arxiv.org/abs/1603.08511);

2. Использованиe в качестве выученных представлений не латентное пространство, а активации других слоев автокодировщика;

3. Добавление ядерной регулизации для уменьшения переобучения расмотренных моделей;

4. Использование более сложных архитектур для классификации с целью получения максмального качества на маленькой обучающей выборке. 